In [16]:
import os
import tarfile
import boto3

# 로컬 모델 파일 경로 및 S3 경로 설정
local_model_path = "/home/ec2-user/SageMaker/best_resnet_ngok_recall3.pth"
compressed_model_path = "/home/ec2-user/SageMaker/best_resnet_model.tar.gz"
bucket_name = "smwu-cv-team-4"  # S3 버킷 이름
s3_model_key = "Resnet_model/best_resnet_model.tar.gz"
s3_model_path = f"s3://{bucket_name}/{s3_model_key}"

# 모델 파일을 TAR.GZ로 압축
with tarfile.open(compressed_model_path, "w:gz") as tar:
    tar.add(local_model_path, arcname="model.pth")
print(f"Model compressed to: {compressed_model_path}")

# S3에 업로드
s3 = boto3.client("s3")
s3.upload_file(compressed_model_path, bucket_name, s3_model_key)
print(f"Compressed model uploaded to: {s3_model_path}")


Model compressed to: /home/ec2-user/SageMaker/best_resnet_model.tar.gz
Compressed model uploaded to: s3://smwu-cv-team-4/Resnet_model/best_resnet_model.tar.gz


In [14]:
from sagemaker.pytorch import PyTorchModel
import sagemaker

# SageMaker 세션 및 역할
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# SageMaker 모델 생성
model = PyTorchModel(
    model_data=s3_model_path,  # 압축된 S3 경로
    role=role,
    entry_point="inference.py",  # 추론 스크립트 경로
    framework_version="1.12",  # PyTorch 버전
    py_version="py38",
    sagemaker_session=sagemaker_session,
)

# 엔드포인트 배포
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",  # 인스턴스 유형
    endpoint_name="team4-resnet-endpoint",  # 엔드포인트 이름
)

print("Endpoint deployed successfully.")


------!Endpoint deployed successfully.


In [3]:
import os

# 현재 작업 디렉토리 확인
current_directory = os.getcwd()
print(f"현재 작업 디렉토리: {current_directory}")


현재 작업 디렉토리: /home/ec2-user/SageMaker


In [4]:
import os
print(os.path.exists("/home/ec2-user/SageMaker/best_resnet_ngok_recall3.pth"))


True


In [5]:
print(os.path.exists("inference.py"))


True


In [3]:
import json
import base64

# 테스트 이미지 읽기
with open("tofu_0003.jpg", "rb") as f:
    image_bytes = f.read()

# JSON 데이터 생성
input_data = {"image": base64.b64encode(image_bytes).decode()}

# JSON 형식으로 저장
with open("test_input.json", "w") as f:
    json.dump(input_data, f)



In [ ]:
import json
import boto3


# 엔드포인트 호출
client = boto3.client("sagemaker-runtime")
response = client.invoke_endpoint(
    EndpointName="team4-resnet-endpoint",
    ContentType="application/json",
    Body=json.dumps(input_data)  # JSON 직렬화된 데이터
)

# 결과 출력
result = json.loads(response["Body"].read().decode())
print(json.dumps(result, indent=4))
